# **98% Accuracy - Chest X-ray Images (Pneumonia or Normal) Classification using CNN**
![](https://media.springernature.com/lw685/springer-static/image/art%3A10.1007%2Fs13369-021-06127-z/MediaObjects/13369_2021_6127_Fig2_HTML.png)


   **98% Accuracy in Predicting if the X-ray is of a Pneumonia Patient or not**

> # Importing and labeling the data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import warnings
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2


warnings.filterwarnings('ignore')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
Train_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train/'
Test_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/test/'
Val_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/val/'


Categories = os.listdir(Train_dir)
print("Train",Categories, "0 is Pneumonia and 1 is Normal")



img_size = 150
def create_data(dir):
    data = []
    for d in os.listdir(dir):
        path_dir = os.path.join(dir,d)
        class_num = Categories.index(d)
        for file in os.listdir(path_dir):
            img_array = cv2.imread(os.path.join(path_dir,file), cv2.IMREAD_GRAYSCALE)
            new_img_array = cv2.resize(img_array, (img_size, img_size))
            data.append([new_img_array, class_num])
    return data
training_data = create_data(Train_dir)
testing_data = create_data(Test_dir)
validation_data = create_data(Val_dir)

> # Shuffling the data so that random samples are provided to the model while training

In [ ]:
import random 

random.shuffle(training_data)

X_train = []
y_train = []

for features,labels in training_data:
    X_train.append(features)
    y_train.append(labels)
print(X_train[:1], y_train[:1])

> # Plotting a few sample images

In [ ]:
import matplotlib.pyplot as plt

for img,label in zip(X_train[:2], y_train[:2]):
    plt.imshow(img, cmap='gray')
    plt.title(Categories[label])
    plt.show()

> # Reshaping and converting X and y to numpy arrays for Training 

In [ ]:
from keras.utils import to_categorical

X = np.array(X_train).reshape(-1, img_size,img_size, 1)
X = X / 255.0
y = np.array(y_train)
print(X[:1], y[:1])

In [ ]:
print(X.shape, y.shape)
X.shape[1:]

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Conv2D, MaxPool2D, Dropout, Flatten, LeakyReLU


Model = Sequential()

Model.add(Conv2D(32, (3,3), padding = 'same', input_shape=X.shape[1:]))
Model.add(Activation('relu'))
Model.add(MaxPool2D(pool_size=(2,2)))

Model.add(Conv2D(64, (3,3), padding = 'same'))
Model.add(Activation('relu'))
Model.add(MaxPool2D(pool_size=(2,2)))

Model.add(Conv2D(96, (3,3), padding = 'same'))
Model.add(Activation('relu'))
Model.add(MaxPool2D(pool_size=(2,2)))
# Model.add(Dropout(0.2))

Model.add(Conv2D(128, (3,3), padding = 'same'))
Model.add(Activation('relu'))
Model.add(MaxPool2D(pool_size=(2,2)))
# Model.add(Dropout(0.2))

Model.add(Conv2D(256, (3,3), padding = 'same'))
Model.add(Activation('relu'))
Model.add(MaxPool2D(pool_size=(2,2)))
# Model.add(Dropout(0.2))

Model.add(Conv2D(320, (3,3), padding = 'same'))
Model.add(Activation('relu'))
Model.add(MaxPool2D(pool_size=(2,2)))
# Model.add(Dropout(0.2))

Model.add(Conv2D(320, (3,3), padding = 'same'))
Model.add(Activation('relu'))
Model.add(MaxPool2D(pool_size=(2,2)))
# Model.add(Dropout(0.2))


Model.add(Flatten())

Model.add(Dense(256))
Model.add(Activation('relu'))
# Model.add(Dropout(0.2))

Model.add(Dense(64))
Model.add(Activation('relu'))
# Model.add(Dropout(0.2))

Model.add(Dense(32))
Model.add(Activation('relu'))

Model.add(Dense(1))
Model.add(Activation('sigmoid'))

Model.summary()

Model.compile(loss = 'binary_crossentropy', optimizer = 'RMSprop', metrics=['accuracy'])

History = Model.fit(X, y, batch_size = 64, epochs = 10,validation_split = 0.1)



In [ ]:
acc = History.history['accuracy']
val_acc = History.history['val_accuracy']

# Retrieve a list of loss results on training and validation data
loss = History.history['loss']
val_loss = History.history['val_loss']


epochs = range(1, len(acc) + 1)


# Plot training and validation accuracy values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot training and validation loss values
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
X_val = []
y_val = []

for X, y in validation_data:
    X_val.append(X)
    y_val.append(y)
    


X_val = np.array(X_val).reshape(-1, img_size,img_size, 1)
X_val = X_val / 255.0
y_val = np.array(y_val)

Val_Eval = Model.evaluate(X_val, y_val, batch_size = 32)
Val_Eval

In [ ]:
X_test = []
y_test = []

for X, y in testing_data:
    X_test.append(X)
    y_test.append(y)
    


X_test = np.array(X_test).reshape(-1, img_size,img_size, 1)
X_test = X_test / 255.0
y_test = np.array(y_test)

Test_Eval = Model.evaluate(X_test, y_test, batch_size = 32)
Test_Eval

In [ ]:
from sklearn.metrics import mean_squared_error
# sklearn.metrics.mean_squared_error
y_pred_test = Model.predict(X_test)
y_pred_val = Model.predict(X_val)

MSE_val = mean_squared_error(y_val, y_pred_val)
MSE_test = mean_squared_error(y_test, y_pred_test)
print("Mean Squared Error")
print("Validation Set: ", MSE_val, "Testing Set: ", MSE_test)
